# COVID-19 Choropleth Maps

This notebook focuses on creating interactive choropleth maps to visualize the global impact of COVID-19 across different countries. We'll create maps for cases, death rates, vaccination coverage, and animated time series.

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Load and Prepare Data

We'll load the OWID COVID-19 dataset and prepare it for creating choropleth maps. We need to ensure we have ISO country codes for mapping and filter the data appropriately.

In [2]:
# Load the COVID-19 dataset
data = pd.read_csv("owid-covid-data.csv")

# Display the first few rows
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [3]:
# Check for ISO codes in the dataset (required for choropleth maps)
print("Available columns for country identification:")
columns = [col for col in data.columns if 'iso' in col.lower() or 'code' in col.lower()]
print(columns)

# Check if we have the iso_code column
if 'iso_code' in data.columns:
    print("\nISO codes sample:")
    print(data['iso_code'].head())
else:
    print("\nWarning: No ISO codes found. We'll need to add them.")

Available columns for country identification:
['iso_code']

ISO codes sample:
0    AFG
1    AFG
2    AFG
3    AFG
4    AFG
Name: iso_code, dtype: object


In [4]:
# Create a snapshot of the most recent data for each country
# This will be used for static choropleth maps

# Convert date to datetime
data['date'] = pd.to_datetime(data['date'])

# Get the most recent date for each country
latest_data = data.sort_values('date').groupby('location').tail(1)

print(f"Total countries in latest data: {len(latest_data)}")
latest_data.head()

Total countries in latest data: 255


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
344539,ESH,Africa,Western Sahara,2022-04-20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,70.26,NaN,576005.0,NaN,NaN,NaN,NaN
228546,OWID_CYN,Asia,Northern Cyprus,2022-12-06,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,382836.0,NaN,NaN,NaN,NaN
180751,MAC,Asia,Macao,2023-04-13,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,84.24,NaN,695180.0,NaN,NaN,NaN,NaN
343153,OWID_WLS,Europe,Wales,2023-07-12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3170000.0,NaN,NaN,NaN,NaN
137002,HKG,Asia,Hong Kong,2023-08-31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,84.86,0.949,7488863.0,NaN,NaN,NaN,NaN


## 2. Create Global Choropleth Map of Total Cases

Let's create our first choropleth map showing the total COVID-19 cases by country.

In [5]:
# Create a choropleth map for total cases
fig_cases = px.choropleth(
    latest_data,
    locations="iso_code",
    color="total_cases",
    hover_name="location",
    color_continuous_scale="Viridis",
    title="Total COVID-19 Cases by Country",
    labels={'total_cases': 'Total Cases'},
    hover_data={
        'total_cases': True,
        'date': True,
        'population': True
    }
)

# Improve the layout
fig_cases.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Total Cases"),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    )
)

# Show the map
fig_cases.show()

## 3. Create Global Choropleth Map of Death Rates

Now we'll visualize the death rate (deaths per case) across different countries to understand where COVID-19 has been most lethal.

In [6]:
# Calculate death rate for each country (as percentage)
latest_data['death_rate'] = (latest_data['total_deaths'] / latest_data['total_cases']) * 100

# Create a choropleth map for death rates
fig_death_rate = px.choropleth(
    latest_data,
    locations="iso_code",
    color="death_rate",
    hover_name="location",
    color_continuous_scale="Reds",
    range_color=[0, 10],  # Adjust range as needed
    title="COVID-19 Death Rate by Country (Deaths per 100 Cases)",
    labels={'death_rate': 'Death Rate (%)'},
    hover_data={
        'total_cases': True,
        'total_deaths': True,
        'death_rate': ':.2f',
        'date': True
    }
)

# Improve the layout
fig_death_rate.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Death Rate (%)"),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    )
)

# Show the map
fig_death_rate.show()

## 4. Create Vaccination Coverage Map

Let's visualize the percentage of the population that has been fully vaccinated across different countries.

In [7]:
# Calculate vaccination percentage
latest_data['vaccination_percentage'] = (latest_data['people_fully_vaccinated'] / latest_data['population']) * 100

# Create a choropleth map for vaccination coverage
fig_vaccination = px.choropleth(
    latest_data,
    locations="iso_code",
    color="vaccination_percentage",
    hover_name="location",
    color_continuous_scale="Blues",
    range_color=[0, 100],
    title="COVID-19 Vaccination Coverage by Country (% of Population Fully Vaccinated)",
    labels={'vaccination_percentage': 'Vaccinated (%)'},
    hover_data={
        'people_fully_vaccinated': True,
        'population': True,
        'vaccination_percentage': ':.2f',
        'date': True
    }
)

# Improve the layout
fig_vaccination.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Vaccinated (%)"),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    )
)

# Show the map
fig_vaccination.show()

## 5. Create Animated Time Series Choropleth

Now, let's create an animated choropleth map that shows how COVID-19 cases have evolved over time.

In [8]:
# Prepare data for animation
# Let's use monthly data to keep the animation smooth
data['month'] = data['date'].dt.strftime('%Y-%m')

# Group by month and country to get monthly snapshots
monthly_data = data.groupby(['month', 'location', 'iso_code'], as_index=False).agg({
    'total_cases': 'max',
    'total_deaths': 'max',
    'date': 'max',
    'population': 'first'
})

# Calculate death rate
monthly_data['death_rate'] = (monthly_data['total_deaths'] / monthly_data['total_cases']) * 100

# Sort by date
monthly_data = monthly_data.sort_values('date')

In [9]:
# Create animated choropleth map for total cases
fig_animated = px.choropleth(
    monthly_data,
    locations="iso_code",
    color="total_cases",
    hover_name="location",
    animation_frame="month",
    color_continuous_scale="Viridis",
    title="Evolution of COVID-19 Cases Over Time",
    labels={'total_cases': 'Total Cases'},
    hover_data={
        'total_cases': True,
        'total_deaths': True,
        'death_rate': ':.2f',
        'date': True
    }
)

# Improve the layout
fig_animated.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Total Cases"),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    )
)

# Configure animation
fig_animated.update_layout(
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

# Show the animated map
fig_animated.show()

## 6. Additional Customization and Interactivity

Let's create a more advanced visualization with multiple metrics displayed on the same map.

In [10]:
# Create a more customized choropleth map with enhanced tooltips
fig_custom = px.choropleth(
    latest_data,
    locations="iso_code",
    color="total_cases_per_million",
    hover_name="location",
    color_continuous_scale=px.colors.sequential.Plasma,
    title="COVID-19 Impact by Country",
    hover_data={
        'total_cases': ':,',
        'total_deaths': ':,',
        'death_rate': ':.2f',
        'vaccination_percentage': ':.2f',
        'total_cases_per_million': ':.2f',
        'date': True
    }
)

# Add custom hover template for better information display
hovertemplate = (
    "<b>%{hovertext}</b><br>" +
    "Date: %{customdata[5]}<br>" +
    "Total Cases: %{customdata[0]}<br>" +
    "Total Deaths: %{customdata[1]}<br>" +
    "Death Rate: %{customdata[2]:.2f}%<br>" +
    "Vaccination Rate: %{customdata[3]:.2f}%<br>" +
    "Cases per Million: %{customdata[4]:.2f}<br>" +
    "<extra></extra>"
)

fig_custom.update_traces(
    customdata=latest_data[['total_cases', 'total_deaths', 'death_rate', 'vaccination_percentage', 
                           'total_cases_per_million', 'date']],
    hovertemplate=hovertemplate
)

# Improve layout with more interactivity
fig_custom.update_layout(
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(title="Cases per Million"),
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth',
        lataxis_showgrid=True,
        lonaxis_showgrid=True
    )
)

# Add a button to switch between cases and death rate
fig_custom.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"z": [latest_data['total_cases_per_million']],
                           "colorbar.title": "Cases per Million"}],
                    label="Cases per Million",
                    method="update"
                ),
                dict(
                    args=[{"z": [latest_data['death_rate']],
                           "colorbar.title": "Death Rate (%)"}],
                    label="Death Rate",
                    method="update"
                ),
                dict(
                    args=[{"z": [latest_data['vaccination_percentage']],
                           "colorbar.title": "Vaccination (%)"}],
                    label="Vaccination Rate",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Show the custom map
fig_custom.show()

## 7. Insights and Conclusions

Based on the choropleth maps we've created, we can observe several patterns in the global COVID-19 data:

1. **Case Distribution**: There's significant variation in total COVID-19 cases across countries, with some regions showing much higher case counts than others.

2. **Death Rates**: Death rates vary considerably by region, which could be influenced by factors like healthcare infrastructure, demographics, testing capacity, and reporting methodologies.

3. **Vaccination Progress**: There are significant disparities in vaccination coverage across different parts of the world.

4. **Temporal Patterns**: The animated map reveals how the pandemic evolved over time, with different waves affecting different regions at different times.

These visualizations provide a powerful way to understand the global impact of COVID-19 and can help inform public health decisions and resource allocation strategies.